# Embeddings(Text, Audio)


In [2]:
import torch
from transformers import AutoProcessor, Data2VecAudioModel
from datasets import load_dataset, Dataset, Audio, Features
from glob import glob
import numpy as np
import pandas as pd
from tqdm import tqdm

print(torch.__version__)

/home/arplab/project/paradeigma/multi_modal/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.13.1+cu117


In [3]:
import pickle

with open('/home/arplab/project/paradeigma/multi_modal/model/data/paradeigma_KEMDY20_annotation_nonmissing.pkl', 'rb') as f:
    annotation_20_nonmissing = pickle.load(f)

In [4]:
annotation_20_nonmissing

,Segment ID,Emotion,Valence,Arousal,emotion_vector,valence_vector,arousal_vector,EDA,TEMP,EDA length,TEMP length,Scaled EDA,Scaled TEMP
0,Sess01_script01_User002M_001,neutral,3.4,2.9,"[10, 0, 0, 0, 0, 0, 0]","[0, 0, 6, 4, 0]","[0, 1, 9, 0, 0]","[-0.06791500000000017, -0.110201, -0.025627999...","[34.81, 34.81, 34.81, 34.79, 34.79, 34.79, 34....",32,32,"[-0.3956120608509403, -0.6460741139849967, -0....","[-0.5852354223355396, -0.5852354223355396, -0...."
1,Sess01_script01_User002M_002,neutral,3.1,2.9,"[10, 0, 0, 0, 0, 0, 0]","[0, 0, 9, 1, 0]","[0, 1, 9, 0, 0]","[0.006407000000000274, -0.021784000000000248, ...","[34.79, 34.79, 34.79, 34.77, 34.77, 34.77, 34....",47,47,"[-0.12237588402818134, -0.09202025752503212, 0...","[-0.7735061037287296, -0.7735061037287296, -0...."
2,Sess01_script01_User002M_003,neutral,3.1,3.0,"[10, 0, 0, 0, 0, 0, 0]","[0, 0, 9, 1, 0]","[0, 0, 10, 0, 0]","[0.07944700000000005, 0.05125599999999997, 0.0...","[34.75, 34.75, 34.75, 34.75, 34.79, 34.79, 34....",33,33,"[0.27229464490531313, 0.1280920921549497, 0.15...","[-1.1500474665150429, -1.1500474665150429, -1...."
3,Sess01_script01_User002M_004,neutral,3.7,3.1,"[9, 1, 0, 0, 0, 0, 0]","[0, 0, 3, 7, 0]","[0, 0, 9, 1, 0]","[0.0038439999999999586, -0.03331699999999982, ...","[34.77, 34.77, 34.77, 34.77, 34.77, 34.77, 34....",45,45,"[-1.6858831480351115, -0.8661805406159092, -0....","[-0.9617767851218528, -0.9617767851218528, -0...."
4,Sess01_script01_User001F_001,neutral,3.8,2.8,"[6, 4, 0, 0, 0, 0, 0]","[0, 0, 3, 6, 1]","[0, 2, 8, 0, 0]","[0.1241910000000006, 0.09474499999999964, 0.03...","[34.55, 34.55, 34.55, 34.55, 34.55, 34.55, 34....",9,9,"[1.8771197424131327, 1.4195480026451441, 0.544...","[-2.4621880639731812, -2.4621880639731812, -2...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12710,Sess40_script06_User079F_043,neutral,3.0,3.5,"[10, 0, 0, 0, 0, 0, 0]","[0, 0, 10, 0, 0]","[0, 0, 5, 5, 0]","[-0.023063999999999973, -0.02050200000000002, ...","[35.07, 35.07, 35.07, 35.07, 35.05, 35.05, 35....",22,22,"[-0.5026723961033952, 0.31424983410693796, -0....","[1.3222897297499479, 1.3222897297499479, 1.322..."
12711,Sess40_script06_User079F_044,neutral,2.9,3.1,"[10, 0, 0, 0, 0, 0, 0]","[0, 1, 9, 0, 0]","[0, 1, 7, 2, 0]","[-0.0064070000000000515, -0.005125000000000046...","[35.05, 35.05, 35.07, 35.07, 35.07, 35.07, 35....",10,10,"[-0.09417143502076045, -0.29842191556206893, -...","[0.8683417878619597, 0.8683417878619597, 1.322..."
12712,Sess40_script06_User079F_045,neutral,2.6,3.6,"[9, 0, 0, 0, 0, 0, 1]","[0, 4, 6, 0, 0]","[0, 0, 4, 6, 0]","[-0.007688000000000028, -0.002561999999999953,...","[35.05, 35.05, 35.05, 35.05, 35.05, 35.05, 35....",27,27,[],"[0.8683417878619597, 0.8683417878619597, 0.868..."
12713,Sess40_script06_User079F_046,neutral,2.7,3.3,"[8, 0, 1, 1, 0, 0, 0]","[0, 3, 7, 0, 0]","[0, 0, 7, 3, 0]","[-0.0064069999999999405, -0.003844000000000069...","[35.05, 35.07, 35.07, 35.07, 35.07, 35.07, 35....",11,11,[],"[0.8683417878619597, 1.3222897297499479, 1.322..."


In [5]:
file_lst = annotation_20_nonmissing['Segment ID']

## Audio Embedding

In [6]:
wav_file_lst = []
for segment in file_lst:
    session = segment[4:6]
    wav_file_lst.append(f'/home/arplab/project/paradeigma/multi_modal/org_KEMDy20/Session{session}/' + segment + '.wav')
wav_file_lst = pd.Series(wav_file_lst)
sampling_rate = 16000


In [7]:
torch.cuda.is_available()

True

In [8]:
check_point_wav_model = 'facebook/data2vec-audio-large-960h'
processor = AutoProcessor.from_pretrained(check_point_wav_model)
model_wav = Data2VecAudioModel.from_pretrained(check_point_wav_model)
model_wav_cuda = Data2VecAudioModel.from_pretrained(check_point_wav_model).to('cuda')

Some weights of the model checkpoint at facebook/data2vec-audio-large-960h were not used when initializing Data2VecAudioModel: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Data2VecAudioModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Data2VecAudioModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at facebook/data2vec-audio-large-960h were not used when initializing Data2VecAudioModel: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Data2VecAudioModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model fr

### audio embedding method

In [9]:
def session_check(segment_id, sess):
    if sess in segment_id:
        return True
    
    else:
        return False

# padding: https://huggingface.co/docs/transformers/pad_truncation
def wav_embeddings_func(wav_file_lst, processor, model):
    dataset = Dataset.from_dict({"wav": wav_file_lst }).cast_column("wav", Audio())
    wav_array = [dataset[i]['wav']['array'] for i in range(len(wav_file_lst))]
    inputs = processor(wav_array, sampling_rate = sampling_rate, return_tensors = 'pt', padding = 'max_length', max_length= 48000, truncation = True)
    with torch.no_grad():
        output = model_wav(**inputs)
    return output

def make_wav_embedding_pickle(file_lst, processor, model, sess_lst):
    
    for session in tqdm(sess_lst):
        wav_embeddings = wav_embeddings_func(file_lst[file_lst.apply(session_check, sess=session)], processor, model)
        wav_embeddings = wav_embeddings['last_hidden_state']
        with open(f'/home/arplab/project/paradeigma/multi_modal/model/data/embedding/KEMDY20/paradeigma_KEMDY20_wav_embedding_{session}.pkl', 'wb') as f:
            pickle.dump(wav_embeddings, f, pickle.HIGHEST_PROTOCOL)
            
        del wav_embeddings
        torch.cuda.empty_cache()
        
def load_wav_embedding_pickle(path_list):
    result = {}
    for path in path_list:
        session = path[-10:-4]
        with open(f'/home/arplab/project/paradeigma/multi_modal/model/data/embedding/KEMDY20/{path}', 'rb') as f:
            wav_embedding = pickle.load(f)
            
        result[session] = wav_embedding
        
    return result

In [12]:
sess_lst = ['Sess20',
       'Sess21', 'Sess22', 'Sess23', 'Sess24', 'Sess25', 'Sess26',
       'Sess27', 'Sess28', 'Sess29', 'Sess30', 'Sess31', 'Sess32',
       'Sess33', 'Sess34', 'Sess35', 'Sess36', 'Sess37', 'Sess38',
       'Sess39', 'Sess40']

In [10]:
sess_lst = annotation_20_nonmissing['Segment ID'].apply(lambda x: x[:6]).unique()

In [14]:
make_wav_embedding_pickle(wav_file_lst, processor, model_wav_cuda, sess_lst)

  0%|          | 0/21 [00:00<?, ?it/s]2023-04-10 00:13:04.518463: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-10 00:13:05.575108: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11/include:/usr/local/cuda-11/lib64::/usr/local/cuda/extras/CUPTI/lib64
2023-04-10 00:13:05.575197: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr

In [19]:
import os

dir_path = '/home/arplab/project/paradeigma/multi_modal/model/data/embedding/KEMDY20'
file_list = os.listdir(dir_path)
wav_path_list = sorted([file for file in file_list if 'KEMDY20_wav' in file ])

embeddings_wav_dict = load_wav_embedding_pickle(wav_path_list)

## Text Embedding
- 참고: https://huggingface.co/docs/datasets/nlp_load
- 참고: https://huggingface.co/docs/transformers/v4.27.2/en/model_doc/data2vec#transformers.Data2VecTextModel
- 참고 한국어 pretrained model: https://huggingface.co/Junmai/KR-Data2VecText-v1

In [52]:
from transformers import PreTrainedTokenizerFast, BartModel
from transformers import AutoModel, AutoTokenizer
import sentencepiece as spm
import pandas as pd
import re


In [48]:
# text data loading
txt_file_lst = []
for segment in file_lst:
    session = segment[4:6]
    txt_file_lst.append(f'/home/arplab/project/paradeigma/multi_modal/org_KEMDy20/Session{session}/' + segment + '.txt')

txt_file_lst = pd.Series(txt_file_lst)

In [53]:
checkpoint = 'Junmai/KR-Data2VecText-v1'
model_txt = AutoModel.from_pretrained(checkpoint) 
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

model_txt.resize_token_embeddings(len(tokenizer))

Embedding(35002, 768)

### text embedding method

In [16]:
def text_embeddings_func(txt_file_lst, tokenizer, model):
    sentences = []

    for i in txt_file_lst:    
        f = open(i, 'r')
        line = f.readline()
        line = re.sub('\n', '', line)
        line = re.sub('  ', ' ', line)
        line = line.rstrip().lstrip()
        sentences.append(line)
        f.close()
    
    inputs = tokenizer(sentences, padding='max_length', max_length = 80, truncation=True, return_tensors="pt")
    with torch.no_grad():
        txt_embeddings= model_txt(**inputs)
    return txt_embeddings['last_hidden_state']

def make_txt_embedding_pickle(file_lst, tokenizer, model, sess_lst):
    
    for session in tqdm(sess_lst):
        txt_embeddings = text_embeddings_func(file_lst[file_lst.apply(session_check, sess=session)], tokenizer, model)
        with open(f'/home/arplab/project/paradeigma/multi_modal/model/data/embedding/KEMDY20/paradeigma_KEMDY20_txt_embedding_{session}.pkl', 'wb') as f:
            pickle.dump(txt_embeddings, f, pickle.HIGHEST_PROTOCOL)
            
        del txt_embeddings
        torch.cuda.empty_cache()
    
def load_txt_embedding_pickle(path_list):
    result = {}
    for path in path_list:
        session = path[-10:-4]
        with open(f'/home/arplab/project/paradeigma/multi_modal/model/data/embedding/KEMDY20/{path}', 'rb') as f:
            txt_embedding = pickle.load(f)
        result[session] = txt_embedding
        
    return result

In [57]:
sess_lst = annotation_20_nonmissing['Segment ID'].apply(lambda x: x[:6]).unique()

make_txt_embedding_pickle(txt_file_lst, tokenizer, model_txt, sess_lst)

100%|██████████| 38/38 [12:56<00:00, 20.44s/it]


In [17]:
import os
import pickle

dir_path = '/home/arplab/project/paradeigma/multi_modal/model/data/embedding/KEMDY20'
file_list = os.listdir(dir_path)
txt_path_list = sorted([file for file in file_list if 'KEMDY20_txt' in file ])

embeddings_txt_dict = load_txt_embedding_pickle(txt_path_list)


In [20]:
embedding_for_dataset = [embeddings_wav_dict, embeddings_txt_dict]

In [21]:
import pickle
with open('/home/arplab/project/paradeigma/multi_modal/model/data/paradeigma_KEMDY20_embedding_for_dataset.pkl', 'wb') as f:
    pickle.dump(embedding_for_dataset, f, pickle.HIGHEST_PROTOCOL)


In [22]:
import pickle
with open('/home/arplab/project/paradeigma/multi_modal/model/data/paradeigma_KEMDY20_embedding_for_dataset.pkl', 'rb') as f:
    embedding_for_dataset = pickle.load(f)
